## flipkart Scrapper using url.csv in flipkart_categories.xlsx

In [1]:
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
import os

### From url to individual page links along with category and img links

In [2]:
url_file = pd.read_csv('url.csv')

In [3]:
url_list = url_file['url'].to_numpy()
cat1 = url_file['Category_1'].to_numpy()
cat2 = url_file['Category_2'].to_numpy()
cat3 = url_file['Category_3'].to_numpy()

In [22]:
links=[]
product_name=[]
start_link = "https://www.flipkart.com"
img_links = []
category1 = []
category2 = []
category3 = []

In [ ]:
i = 1
for url,a,b,c in zip(url_list,cat1,cat2,cat3):
    print(i)
    try:
        req = requests.get(url)
        content = BeautifulSoup(req.content,'html.parser')
        #works for iteration 1 with description
        data = content.find_all('div',{'class':'_4ddWXP'}) 
        #works for iteration 2 usually such catergories have no description, specification, heading
        data2 = content.find_all('div',{'class':'_1xHGtK _373qXS'})
        #works for iteration 3 usually with review blocks inside the product block div. For products whose standard display of products is not available. In case available we do not require this as these will already be covered in above 2.
        data3 = []
        if not len(data2) and not len(data):
          data3 = content.find_all('div',{'class':'_1Ni40J'})
        for items in data:
          res_link = items.find('a')['href']
          img_link = items.find('img')['src']
          links.append(start_link+res_link)
          img_links.append(img_link)
          category1.append(a)
          category2.append(b)
          category3.append(c)

        for items in data2:
          res_link = items.find('a')['href']
          img_link = items.find('img')['src']
          links.append(start_link+res_link)
          img_links.append(img_link)
          category1.append(a)
          category2.append(b)
          category3.append(c)

        for items in data3:
          res_link = items.find('a')['href']
          img_link = items.find('img')['src']
          links.append(start_link+res_link)
          img_links.append(img_link)
          category1.append(a)
          category2.append(b)
          category3.append(c)
    except:
        print(f'sorry brother for {url}')

### Page Parsing

In [41]:
#product name string without processing Brand name is before \x0 but not in all

#rating string needs to be converted to number

#purchase price string with rupee or dollar sign and commas

#actual price string with rupee or dollar sign and commas

#discount can be calculated (Not Included for now)

#img url captured before in above step

#rating of seller string decimal

#seller name string

#item description string ########## some products might not have description

#Highlights string       ############### some products might not have highlights

#specifications values only string

0        
1        
2        
3        
4        
       ..
1833     
1834     
1835     
1836     
1837     
Name: specifications, Length: 1838, dtype: object

In [ ]:
headings = []
ratings = []
sell_prices = []
mrps = []
sellers = []
s_ratings = []
descriptions = []
highlights = []
specifications = []
i=1
for link in links:
  i=i+1
  if i%20==0:
        print(i)
  req_page= requests.get(link) 
  content_page = req_page.content
  data_page = BeautifulSoup(content_page,'html.parser')
  try:
    heading = data_page.find('h1').text
  except:
    heading = ' '
  headings.append(heading)
  try:
    rating = data_page.find('div',{'class':'_3LWZlK'}).text
  except:
    rating = ' '
  ratings.append(rating)
  try:
    sell_price = data_page.find('div',{'class':'_30jeq3 _16Jk6d'}).text
  except:
    sell_price = ' '
  sell_prices.append(sell_price)
  try:
    mrp = data_page.find('div',{'class':'_3I9_wc _2p6lqe'}).text
  except:
    mrp = ' '
  mrps.append(mrp)
  try:
    s = data_page.find('div',{'class':'_1RLviY'}).text
    s=s[:s.find(data_page.find('div',{'class':'_3LWZlK _1D-8OL'}).text)]
  except:
    s = ' '
  sellers.append(s)
  try:
    s_rating = data_page.find('div',{'class':'_3LWZlK _1D-8OL'}).text
  except:
    s_rating = ' '
  s_ratings.append(s_rating)
  try:
    description = data_page.find('div',{'class':'_1mXcCf'}).text
  except:
    description = ' '
  descriptions.append(description)
  try:
    highlight = ''
    for li in data_page.find('div',{'class':'_2418kt'}).find_all('li'):
      highlight= highlight + ' ' +li.text
  except:
    hightlight = ' '
  highlights.append(highlight[1:])
  try:
    specification = ''
    for li in data_page.find('div',{'class':'_3k-BhJ'}).find_all('td'):
      specification= specification_val + ' ' +li.text
  except:
    specification = ' '
  specifications.append(specification[1:])

In [33]:
dataset = pd.DataFrame(
    {
     'category_1':category1,
     'category_2':category2,
     'category_3':category3,
     'title':headings,
     'product_rating':ratings,
     'selling_price':sell_prices,
     'mrp':mrps,
     'seller_name':sellers,
     'seller_rating':s_ratings,
     'description':descriptions,
     'highlights':highlights,
     'specifications':specifications,
     'image_links':img_links
    }
)

In [34]:
dataset.to_csv('poc.csv',index = False)